### Implementing randomforest with pipeline and hyperparameter tunning

* using randomforest classifier we are predicting which time of the it is day 

In [1]:
import seaborn as sns

In [2]:
df=sns.load_dataset('tips')

In [3]:
df

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4
...,...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,Dinner,3
240,27.18,2.00,Female,Yes,Sat,Dinner,2
241,22.67,2.00,Male,Yes,Sat,Dinner,2
242,17.82,1.75,Male,No,Sat,Dinner,2


In [4]:
df.isnull().sum()

total_bill    0
tip           0
sex           0
smoker        0
day           0
time          0
size          0
dtype: int64

In [5]:
df.describe()

,total_bill,tip,size
count,244.000000,244.000000,244.000000
mean,19.785943,2.998279,2.569672
std,8.902412,1.383638,0.951100
min,3.070000,1.000000,1.000000
25%,13.347500,2.000000,2.000000
50%,17.795000,2.900000,2.000000
75%,24.127500,3.562500,3.000000
max,50.810000,10.000000,6.000000


In [6]:
df['day'].unique

<bound method Series.unique of 0       Sun
1       Sun
2       Sun
3       Sun
4       Sun
       ... 
239     Sat
240     Sat
241     Sat
242     Sat
243    Thur
Name: day, Length: 244, dtype: category
Categories (4, object): ['Thur', 'Fri', 'Sat', 'Sun']>

In [7]:
df['time'].unique

<bound method Series.unique of 0      Dinner
1      Dinner
2      Dinner
3      Dinner
4      Dinner
        ...  
239    Dinner
240    Dinner
241    Dinner
242    Dinner
243    Dinner
Name: time, Length: 244, dtype: category
Categories (2, object): ['Lunch', 'Dinner']>

In [8]:
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()
df['time']=encoder.fit_transform(df['time'])

In [9]:
df['time'].unique()

array([0, 1])

In [10]:
x=df.drop(labels=['time'],axis=1)
y=df['time']
# y=df.time

In [11]:
x

,total_bill,tip,sex,smoker,day,size
0,16.99,1.01,Female,No,Sun,2
1,10.34,1.66,Male,No,Sun,3
2,21.01,3.50,Male,No,Sun,3
3,23.68,3.31,Male,No,Sun,2
4,24.59,3.61,Female,No,Sun,4
...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,3
240,27.18,2.00,Female,Yes,Sat,2
241,22.67,2.00,Male,Yes,Sat,2
242,17.82,1.75,Male,No,Sat,2


In [12]:
y

0      0
1      0
2      0
3      0
4      0
      ..
239    0
240    0
241    0
242    0
243    0
Name: time, Length: 244, dtype: int64

In [13]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y, test_size=0.23,random_state=43)

In [14]:
x_train

,total_bill,tip,sex,smoker,day,size
59,48.27,6.73,Male,No,Sat,4
128,11.38,2.00,Female,No,Thur,2
116,29.93,5.07,Male,No,Sun,4
155,29.85,5.14,Female,No,Sun,5
210,30.06,2.00,Male,Yes,Sat,3
...,...,...,...,...,...,...
58,11.24,1.76,Male,Yes,Sat,2
21,20.29,2.75,Female,No,Sat,2
49,18.04,3.00,Male,No,Sun,2
64,17.59,2.64,Male,No,Sat,3


In [15]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer ##    for missing valuw
from sklearn.preprocessing import StandardScaler # for feature scaling
from sklearn.preprocessing import OneHotEncoder # for categorical to numerical
from sklearn.compose import ColumnTransformer 

In [16]:
categorical_cols=['sex'	,'smoker',	'day']
numerical_cols=['total_bill',	'tip'	,'size']

In [17]:
num_pipe=Pipeline(
    steps=[
         ('impute', SimpleImputer(strategy='median')),##missing value
        ('scaler', StandardScaler())
        
    ]
)



In [18]:
cate_pipe=Pipeline(
    steps=[
         ('impute', SimpleImputer(strategy='most_frequent')),##missing value
        ('onehotencoder',OneHotEncoder())   
    ]
)

In [19]:
preprocess=ColumnTransformer([
    ('num_pipe',num_pipe,numerical_cols),
    ('cate_pipe',cate_pipe,categorical_cols)
])

In [20]:
x_train=preprocess.fit_transform(x_train)
x_test=preprocess.transform(x_test)

In [21]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

In [22]:
#model automation
models={
    'random_for_class':RandomForestClassifier(),
    'logistic':LogisticRegression(),
    'decision_tree_class':DecisionTreeClassifier()
}

In [23]:
from sklearn.metrics import accuracy_score

In [24]:
def evaluate_model(x_train,y_train,x_test,y_test,models):
    report={}
    for i in range(len(models)):
        model=list(models.values())[i]
        #train_model
        model.fit(x_train,y_train)
        #prediction of test data
        y_pred=model.predict(x_test)
        
        #get_accuracy_score
        test_model_score=accuracy_score(y_test,y_pred)
        
        report[list(models.keys())[i]] =  test_model_score
        
    return report
        

In [25]:
evaluate_model(x_train,y_train,x_test,y_test,models)

{'random_for_class': 0.9122807017543859,
 'logistic': 0.9122807017543859,
 'decision_tree_class': 0.8947368421052632}

In [35]:
classifier=RandomForestClassifier()

In [36]:
## Hypeparameter Tuning
params={'max_depth':[3,5,10,None],
              'n_estimators':[100,200,300],
               'criterion':['gini','entropy']
              }

In [37]:
from sklearn.model_selection import RandomizedSearchCV

In [40]:
cv=RandomizedSearchCV(classifier,param_distributions=params,scoring='accuracy',cv=5,verbose=3)
cv.fit(x_train,y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5] END criterion=entropy, max_depth=3, n_estimators=100;, score=1.000 total time=   0.2s
[CV 2/5] END criterion=entropy, max_depth=3, n_estimators=100;, score=0.947 total time=   0.2s
[CV 3/5] END criterion=entropy, max_depth=3, n_estimators=100;, score=0.973 total time=   0.2s
[CV 4/5] END criterion=entropy, max_depth=3, n_estimators=100;, score=1.000 total time=   0.2s
[CV 5/5] END criterion=entropy, max_depth=3, n_estimators=100;, score=1.000 total time=   0.2s
[CV 1/5] END criterion=entropy, max_depth=3, n_estimators=200;, score=1.000 total time=   0.4s
[CV 2/5] END criterion=entropy, max_depth=3, n_estimators=200;, score=0.947 total time=   0.4s
[CV 3/5] END criterion=entropy, max_depth=3, n_estimators=200;, score=0.973 total time=   0.4s
[CV 4/5] END criterion=entropy, max_depth=3, n_estimators=200;, score=1.000 total time=   0.4s
[CV 5/5] END criterion=entropy, max_depth=3, n_estimators=200;, score=1.000 total ti

RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(),
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': [3, 5, 10, None],
                                        'n_estimators': [100, 200, 300]},
                   scoring='accuracy', verbose=3)

In [41]:
cv.best_params_

{'n_estimators': 100, 'max_depth': 3, 'criterion': 'entropy'}

In [42]:
y_pred_cv=cv.predict(x_test)

In [43]:
cv_score=accuracy_score(y_test,y_pred_cv)

In [44]:
cv_score

0.9122807017543859